In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_union, make_pipeline, Pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE, RFECV

from fenginering.indicator import *
from fenginering.transformer import *
from data.get_data import *

from fenginering.function import *

%load_ext autoreload
%autoreload 2

In [2]:
start = "2022"
end = "2023"
data = get_data('BTC', '1d').loc[start : end]
data.drop(columns = ['symbol'], inplace = True)
data.columns = data.columns.astype(str)
data.shape

(449, 5)

# Combinaison de MA

In [3]:
ma_list = [3, 7, 10, 14, 21]

In [4]:
EMA_transformer = one_params_transformer(ma_list, ema)
CROSS_OVER_transformer = FunctionTransformer(crossover)
CROSS_OVER_DIST_transformer = FunctionTransformer(crossover_dist)
DROPNA_transformer = dropna_transformer()

In [5]:
ma_pipeline_1 = make_union(
    EMA_transformer,
    CROSS_OVER_transformer,
    CROSS_OVER_DIST_transformer
)

ma_pipeline = make_pipeline(
    ma_pipeline_1,
    DROPNA_transformer
)

ma_pipeline

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('functiontransformer-1',
                                                 FunctionTransformer(func=<function one_params at 0x0000026A0A917CA0>,
                                                                     kw_args={'funct': <function ema at 0x0000026A08031F70>,
                                                                              'params': [3,
                                                                                         7,
                                                                                         10,
                                                                                         14,
                                                                                         21]})),
                                                ('functiontransformer-2',
                                                 FunctionTransformer(func=<function crossover at 0x0000026A0ADD9790>)),
                                                ('functiontransformer-3',
                                                 FunctionTransformer(func=<function crossover_dist at 0x0000026A0BDD91F0>))])),
                ('functiontransformer',
                 FunctionTransformer(func=<function dropna_transformer.<locals>.<lambda> at 0x0000026A771D5670>))])

In [6]:
X = ma_pipeline.fit_transform(data)

In [7]:
X

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
20,38911.324719,40614.144121,41251.932937,41896.051119,42779.995946,1.0,0.0,0.0,1.0,0.0,...,2.010302,1.871181,1.895882,3.184338,1.862298,1.886747,3.162065,2.028353,3.507330,3.438198
21,36991.372360,39228.463090,40128.203312,40986.100303,42079.216314,1.0,0.0,0.0,1.0,0.0,...,2.010698,1.933124,1.962303,3.482386,1.923247,1.952116,3.456110,2.031270,3.660296,3.579631
22,36617.961180,38482.484818,39422.084528,40353.893596,41548.792104,1.0,0.0,0.0,1.0,1.0,...,2.040705,1.986587,2.033450,2.262553,1.947999,1.993029,2.213171,2.047500,2.279718,2.221688
23,36639.155590,38026.951113,38919.950978,39861.421117,41104.388276,1.0,0.0,0.0,1.0,1.0,...,2.036018,1.908196,2.011472,3.535685,1.876622,1.976308,3.447530,2.113715,3.792000,3.506925
24,36798.737795,37759.793335,38563.290800,39474.340968,40727.472978,1.0,0.0,0.0,1.0,1.0,...,2.024135,1.973831,2.008128,2.342933,1.950882,1.984370,2.311286,2.035218,2.379020,2.332106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,27784.855237,26787.506484,26118.259796,25474.603158,24789.246570,1.0,0.0,0.0,1.0,1.0,...,2.026032,1.985067,2.014015,16.186694,1.960074,1.988288,15.801378,2.029388,16.416921,15.976791
445,27517.912618,26903.372363,26324.207106,25711.452071,25013.039609,1.0,0.0,0.0,1.0,0.0,...,2.027047,1.946420,1.969516,8.973350,1.921496,1.943984,8.763372,2.024403,9.424746,9.224053
446,27906.661309,27251.381773,26682.607632,26055.979795,25311.436917,1.0,0.0,0.0,1.0,1.0,...,2.055008,1.994643,2.038327,5.720918,1.942783,1.984188,5.474769,2.043918,5.746342,5.546660
447,27680.565655,27302.153829,26822.946244,26242.445155,25506.258107,1.0,0.0,0.0,1.0,0.0,...,2.002788,1.954218,1.970280,4.047916,1.951565,1.967582,4.039443,2.016832,4.194151,4.141276
